## Installing Essential Dependencies

In [10]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install tensorflow

## Loading Data \& Pre_Processing

In [11]:
import pandas as pd
import numpy as np
df = pd.read_csv('BADSS_testing_data_2.csv')
# df = pd.read_csv('BADSS_training_data_Ours.csv')
df.columns = df.columns.str.strip()
df

,Date,Symbol,Maturity,Strike,Bid Price,Bid Size,Ask Price,Ask Size,Undl Price,date_id,...,exposure_23,exposure_24,exposure_25,exposure_26,exposure_27,exposure_28,exposure_29,exposure_30,exposure_31,exposure_32
0,2024-07-31,SPY,2024-08-01,551.0,2.67,2222,2.72,2199,550.81,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-07-31,SPY,2024-08-01,552.0,2.14,1782,2.18,2337,550.81,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-07-31,SPY,2024-08-01,553.0,1.67,1342,1.71,2476,550.81,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-07-31,SPY,2024-08-01,554.0,1.26,1197,1.29,2928,550.81,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-07-31,SPY,2024-08-01,555.0,0.92,1348,0.95,3695,550.81,1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25638,2024-08-30,QQQ,2024-09-13,550.0,0.00,0,0.02,7920,476.27,23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25639,2024-08-30,QQQ,2024-09-13,555.0,0.00,0,0.02,8946,476.27,23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25640,2024-08-30,QQQ,2024-09-13,560.0,0.00,0,0.02,9851,476.27,23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25641,2024-08-30,QQQ,2024-09-13,565.0,0.00,0,0.02,10932,476.27,23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df["Date"] = pd.to_datetime(df["Date"])
df["Maturity"] = pd.to_datetime(df["Maturity"])

# Combine unique dates from "Date" and "Maturity", sort them, and remove duplicates
unique_dates = pd.concat([df["Date"], df["Maturity"]]).sort_values().unique()
# Create a mapping from each date to a unique sequential id (starting from 1)
date_mapping = {date: i+1 for i, date in enumerate(unique_dates)}

# Map the "Date" and "Maturity" columns to their corresponding ids using the mapping
df["date_id"] = df["Date"].map(date_mapping)
df["maturity_id"] = df["Maturity"].map(date_mapping)

maturity_strike_symbol_dict = (
    df.groupby(["maturity_id", "Strike", "Symbol"])["date_id"]
    .apply(set)
    .to_dict()
)

list(maturity_strike_symbol_dict.items())[990:1000]

[((7, 474.0, 'QQQ'), {1, 2, 3, 4, 5, 6}),
 ((7, 475.0, 'QQQ'), {1, 2, 3, 4, 5, 6}),
 ((7, 476.0, 'QQQ'), {1, 2, 3, 4, 5, 6}),
 ((7, 477.0, 'QQQ'), {1, 2, 3, 4, 5, 6}),
 ((7, 478.0, 'QQQ'), {1, 2, 3, 4, 5, 6}),
 ((7, 479.0, 'QQQ'), {1, 2, 3, 4, 5, 6}),
 ((7, 480.0, 'QQQ'), {1, 2, 3, 4, 5, 6}),
 ((7, 481.0, 'QQQ'), {1, 2, 3, 4, 5, 6}),
 ((7, 482.0, 'QQQ'), {1, 2, 3, 4, 5, 6}),
 ((7, 483.0, 'QQQ'), {1, 2, 3, 4, 5, 6})]

In [13]:
for iterdate in range(1, 23):  # Loop over date_id values from 1 to 22
    current_exposure_col = f"exposure_{iterdate}"  # Get the exposure column for the current date_id
    current_PnL_col_ = f"PnL_{iterdate}"  # Get the PnL column for the current date_id

    if current_exposure_col in df.columns:  # Check if the exposure column exists
        # Compute denominator using the PnL column, add a small offset to avoid division by zero
        denominator = -df[current_PnL_col_] + 0.0001
        denominator = denominator.replace(0, np.nan)  # Replace zeros with NaN to avoid division errors

        # For rows with the current date_id, calculate the ratio of exposure to the denominator
        df.loc[df["date_id"] == iterdate, "exposure_1_ratio"] = df.loc[df["date_id"] == iterdate, current_exposure_col] / denominator

# Display a slice of the DataFrame for inspection
df[1000:1010]


,Date,Symbol,Maturity,Strike,Bid Price,Bid Size,Ask Price,Ask Size,Undl Price,date_id,...,exposure_25,exposure_26,exposure_27,exposure_28,exposure_29,exposure_30,exposure_31,exposure_32,maturity_id,exposure_1_ratio
1000,2024-08-02,SPY,2024-08-08,550.0,0.53,2599,0.59,14484,532.9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.0
1001,2024-08-02,SPY,2024-08-08,551.0,0.43,3546,0.48,16929,532.9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.0
1002,2024-08-02,SPY,2024-08-08,552.0,0.35,4869,0.40,17899,532.9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.0
1003,2024-08-02,SPY,2024-08-08,553.0,0.28,6568,0.33,17393,532.9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.0
1004,2024-08-02,SPY,2024-08-08,554.0,0.23,8268,0.27,16887,532.9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.0
1005,2024-08-02,SPY,2024-08-08,555.0,0.18,8235,0.22,15918,532.9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.0
1006,2024-08-02,SPY,2024-08-08,556.0,0.13,8169,0.17,13979,532.9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.0
1007,2024-08-02,SPY,2024-08-08,557.0,0.10,8103,0.14,12041,532.9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.0
1008,2024-08-02,SPY,2024-08-08,558.0,0.08,8038,0.11,10102,532.9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.0
1009,2024-08-02,SPY,2024-08-08,559.0,0.05,7973,0.09,8163,532.9,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.0


## Define the labelling function

In [14]:
def custom_sort_key(row):
    """
    Computes a sort key based on the ratio of exposure to PnL for the row's corresponding date_id.
    The key groups rows into three categories based on the sign of the ratio and returns a combined value.
    """
    d = int(row["date_id"])
    exposure_val = row.get(f"exposure_{d}", np.nan)
    pnl_val = row.get(f"PnL_{d}", np.nan)
    
    # Avoid division by zero
    if pnl_val == 0 or np.isnan(pnl_val):
        ratio = exposure_val / (pnl_val + 1e-3)
    else:
        ratio = exposure_val / pnl_val

    if ratio > 0:
        group = 0
        subkey = abs(ratio)
    elif ratio < 0:
        group = 1
        subkey = -abs(ratio)
    else:
        group = 2
        subkey = 0

    M = 1e5  # Large constant to separate groups
    return group * M + subkey


In [15]:
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Generate custom labels
df["label"] = df.apply(custom_sort_key, axis=1)

# Select features and encode 'Symbol'
features = ['date_id', 'maturity_id', 'Symbol', 'Strike', 'Bid Price', 'Bid Size', 'Ask Price', 'Ask Size']
le = LabelEncoder()
df["Symbol_enc"] = le.fit_transform(df["Symbol"])
features = ['date_id', 'maturity_id', 'Symbol_enc', 'Strike', 'Bid Price', 'Bid Size', 'Ask Price', 'Ask Size']

# Build feature matrix and target vector
X = df[features].values
y = df["label"].values

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


## Performance of our best labels

In [16]:
holdings = {}
df_date = [1]
total_PnL = 0

# Collect data for each date_id from 1 to 22
for iterdate in range(1, 23):
    df_date.append(df[df["date_id"] == iterdate])

for iterdate in range(1, 23):
    exposure = 0
    PnL = 0
    buy_dict = {}
    sell_dict = {}

    current_exposure_col = f"exposure_{iterdate}"
    current_pnl_col = f"PnL_{iterdate - 1}"
    current_pnl_col_1 = f"PnL_{iterdate}"


    if iterdate > 1:
        for hkey in list(holdings.keys()):
            filtered_row = df[
                (df["maturity_id"] == hkey[0]) &
                (df["date_id"] == iterdate - 1) &
                (df["Strike"] == hkey[1]) &
                (df["Symbol"] == hkey[2])
            ]
            selling_row = df[
                (df["maturity_id"] == hkey[0]) &
                (df["date_id"] == iterdate) &
                (df["Strike"] == hkey[1]) &
                (df["Symbol"] == hkey[2])
            ]
            if not selling_row.empty and current_exposure_col in selling_row.columns:
                sell_dict[hkey] = min(selling_row["Bid Size"].values[0], holdings[hkey])
                exposure += selling_row[current_exposure_col].values[0] * (holdings[hkey] - sell_dict[hkey])
            else:
                sell_dict[hkey] = 0

    # Trading: Buy options with non-profitable PnL to meet exposure target
    if exposure >= 100000:
        new_holdings = {}
        # Process sell orders
        for key, sell_qty in sell_dict.items():
            if key in holdings:
                holdings[key] -= sell_qty
                if holdings[key] <= 0:
                    del holdings[key]
        # Process buy orders
        for key, buy_qty in buy_dict.items():
            if key in holdings:
                holdings[key] += buy_qty
            else:
                holdings[key] = buy_qty
        # Calculate PnL for the day
        for hkey in list(holdings.keys()):
            filtered_row = df[
                (df["maturity_id"] == hkey[0]) &
                (df["date_id"] == iterdate) &
                (df["Strike"] == hkey[1]) &
                (df["Symbol"] == hkey[2])
            ]
            if not filtered_row.empty and current_pnl_col_1 in filtered_row.columns:
                PnL += holdings[hkey] * filtered_row[current_pnl_col_1].values[0]
        for key, qty in holdings.items():
            maturity_id, strike, symbol = key
            next_key = (maturity_id, strike, symbol)
            if maturity_id > iterdate + 1:
                new_holdings[next_key] = qty  
        holdings = new_holdings.copy()
        total_PnL += PnL
        continue

    # If exposure is below target, buy additional options
    df_temp = df_date[iterdate].copy()
    df_temp["priority"] = df_temp.apply(custom_sort_key, axis=1)
    df_sorted = df_temp.sort_values(by="priority", ascending=True)

    for index, row in df_sorted.iterrows():
        key = (row["maturity_id"], row["Strike"], row["Symbol"])
        ask_size = row["Ask Size"]
        max_buy_qty = ask_size
        if key in sell_dict:
            additional_needed = 100000 - exposure
            if row[current_exposure_col] != 0:
                required_reduce_qty = int(np.ceil(additional_needed / row[current_exposure_col]))
            else:
                required_reduce_qty = 0
                continue
            if required_reduce_qty <= sell_dict[key]:
                reduce_qty = required_reduce_qty
                sell_dict[key] -= reduce_qty
                if sell_dict[key] == 0:
                    del sell_dict[key]
                exposure += reduce_qty * row[current_exposure_col]
                break
            else:
                reduce_qty = sell_dict[key]
                del sell_dict[key]
                exposure += reduce_qty * row[current_exposure_col]

        additional_needed = 100000 - exposure
        if row[current_exposure_col] != 0:
            required_buy_qty = int(np.ceil(additional_needed / row[current_exposure_col]))
        else:
            continue
        if required_buy_qty <= max_buy_qty:
            buy_qty = required_buy_qty
            exposure += buy_qty * row[current_exposure_col]
            buy_dict[key] = buy_dict.get(key, 0) + buy_qty
            break
        else:
            buy_qty = max_buy_qty
            exposure += buy_qty * row[current_exposure_col]
            buy_dict[key] = buy_dict.get(key, 0) + buy_qty

    # Update holdings after trades
    new_holdings = {}
    for key, sell_qty in sell_dict.items():
        if key in holdings:
            holdings[key] -= sell_qty
            if holdings[key] <= 0:
                del holdings[key]
    for key, buy_qty in buy_dict.items():
        if key in holdings:
            holdings[key] += buy_qty
        else:
            holdings[key] = buy_qty

    # Calculate PnL for current day
    for hkey in list(holdings.keys()):
        filtered_row = df[
            (df["maturity_id"] == hkey[0]) &
            (df["date_id"] == iterdate) &
            (df["Strike"] == hkey[1]) &
            (df["Symbol"] == hkey[2])
        ]
        if not filtered_row.empty and current_pnl_col_1 in filtered_row.columns:
            PnL += holdings[hkey] * filtered_row[current_pnl_col_1].values[0]
    for key, qty in holdings.items():
        maturity_id, strike, symbol = key
        next_key = (maturity_id, strike, symbol)
        if maturity_id > iterdate + 1:
            new_holdings[next_key] = qty
    holdings = new_holdings.copy()

    total_PnL += PnL

print("The total PnL is:",total_PnL *100)


The total PnL is: 22560856.999999993


## Performance of baseline

In [17]:
# Baseline simulation to compute mean PnL over 25 iterations
mean_PnL = 0
for i in range(25):
    holdings = {}
    df_date = [1]
    total_PnL = 0
    for iterdate in range(1, 23):
        df_date.append(df[df["date_id"] == iterdate])
        
    for iterdate in range(1, 23):
        exposure = 0
        PnL = 0
        buy_dict = {}
        sell_dict = {}
        
        current_exposure_col = f"exposure_{iterdate}"
        current_pnl_col = f"PnL_{iterdate - 1}"
        current_pnl_col_1 = f"PnL_{iterdate}"
        
        if iterdate > 1:
            for hkey in list(holdings.keys()):
                filtered_row = df[
                    (df["maturity_id"] == hkey[0]) &
                    (df["date_id"] == iterdate - 1) &
                    (df["Strike"] == hkey[1]) &
                    (df["Symbol"] == hkey[2])
                ]
                selling_row = df[
                    (df["maturity_id"] == hkey[0]) &
                    (df["date_id"] == iterdate) &
                    (df["Strike"] == hkey[1]) &
                    (df["Symbol"] == hkey[2])
                ]
                if not selling_row.empty and current_exposure_col in selling_row.columns:
                    sell_dict[hkey] = min(selling_row["Bid Size"].values[0], holdings[hkey])
                    exposure += selling_row[current_exposure_col].values[0] * (holdings[hkey] - sell_dict[hkey])
                else:
                    sell_dict[hkey] = 0
        
        # If target exposure reached, update holdings and calculate PnL
        if exposure >= 100000:
            new_holdings = {}
            for key, sell_qty in sell_dict.items():
                if key in holdings:
                    holdings[key] -= sell_qty
                    if holdings[key] <= 0:
                        del holdings[key]
            for key, buy_qty in buy_dict.items():
                if key in holdings:
                    holdings[key] += buy_qty
                else:
                    holdings[key] = buy_qty
            for hkey in list(holdings.keys()):
                filtered_row = df[
                    (df["maturity_id"] == hkey[0]) &
                    (df["date_id"] == iterdate) &
                    (df["Strike"] == hkey[1]) &
                    (df["Symbol"] == hkey[2])
                ]
                if not filtered_row.empty and current_pnl_col_1 in filtered_row.columns:
                    PnL += holdings[hkey] * filtered_row[current_pnl_col_1].values[0]
            for key, qty in holdings.items():
                maturity_id, strike, symbol = key
                next_key = (maturity_id, strike, symbol)
                if maturity_id > iterdate + 1:
                    new_holdings[next_key] = qty  
            holdings = new_holdings.copy()
            total_PnL += PnL
            continue
        
        # If exposure below target, buy additional options
        df_temp = df_date[iterdate].copy()
        df_sorted = df_temp.sample(frac=1).reset_index(drop=True)
        
        for index, row in df_sorted.iterrows():
            key = (row["maturity_id"], row["Strike"], row["Symbol"])
            ask_size = row["Ask Size"]
            max_buy_qty = ask_size
            if key in sell_dict:
                additional_needed = 100000 - exposure
                if row[current_exposure_col] != 0:
                    required_reduce_qty = int(np.ceil(additional_needed / row[current_exposure_col]))
                else:
                    required_reduce_qty = 0
                    continue
                if required_reduce_qty <= sell_dict[key]:
                    reduce_qty = required_reduce_qty
                    sell_dict[key] -= reduce_qty
                    if sell_dict[key] == 0:
                        del sell_dict[key]
                    exposure += reduce_qty * row[current_exposure_col]
                    break
                else:
                    reduce_qty = sell_dict[key]
                    del sell_dict[key]
                    exposure += reduce_qty * row[current_exposure_col]
            
            additional_needed = 100000 - exposure
            if row[current_exposure_col] != 0:
                required_buy_qty = int(np.ceil(additional_needed / row[current_exposure_col]))
            else:
                continue
            if required_buy_qty <= max_buy_qty:
                buy_qty = required_buy_qty
                exposure += buy_qty * row[current_exposure_col]
                buy_dict[key] = buy_dict.get(key, 0) + buy_qty
                break
            else:
                buy_qty = max_buy_qty
                exposure += buy_qty * row[current_exposure_col]
                buy_dict[key] = buy_dict.get(key, 0) + buy_qty
        
        # Update holdings with executed trades
        new_holdings = {}
        for key, sell_qty in sell_dict.items():
            if key in holdings:
                holdings[key] -= sell_qty
                if holdings[key] <= 0:
                    del holdings[key]
        for key, buy_qty in buy_dict.items():
            if key in holdings:
                holdings[key] += buy_qty
            else:
                holdings[key] = buy_qty
        
        # Calculate PnL for current day
        for hkey in list(holdings.keys()):
            filtered_row = df[
                (df["maturity_id"] == hkey[0]) &
                (df["date_id"] == iterdate) &
                (df["Strike"] == hkey[1]) &
                (df["Symbol"] == hkey[2])
            ]
            if not filtered_row.empty and current_pnl_col_1 in filtered_row.columns:
                PnL += holdings[hkey] * filtered_row[current_pnl_col_1].values[0]
        
        for key, qty in holdings.items():
            maturity_id, strike, symbol = key
            next_key = (maturity_id, strike, symbol)
            if maturity_id > iterdate + 1:
                new_holdings[next_key] = qty
        
        holdings = new_holdings.copy()
        total_PnL += PnL
        
    mean_PnL += total_PnL

mean_PnL /= 25
print("The mean total PnL of baseline is:",mean_PnL *100)


The mean total PnL of baseline is: -11299563.119999997


## Performance of our Model

In [18]:
from tensorflow.keras.models import load_model
model = load_model("trained_model.h5")
holdings = {}
total_PnL = 0
df_date = [1]
for iterdate in range(1, 23):
    df_date.append(df[df["date_id"] == iterdate])

daily_records = []

for iterdate in range(1, 23):
    exposure = 0
    PnL = 0
    buy_dict = {}
    sell_dict = {}

    current_exposure_col = f"exposure_{iterdate}"  # Dynamically select exposure column
    current_pnl_col = f"PnL_{iterdate - 1}"         # Select previous day's PnL column
    current_pnl_col_1 = f"PnL_{iterdate}"             # Select current day's PnL column

    print(f"\n === Date {iterdate} ===")
    print("Starting new day, current holdings:")
    print(holdings)

    if iterdate > 1:
        for hkey in list(holdings.keys()):
            filtered_row = df[
                (df["maturity_id"] == hkey[0]) &
                (df["date_id"] == iterdate - 1) &
                (df["Strike"] == hkey[1]) &
                (df["Symbol"] == hkey[2])
            ]
            # Calculate sell quantity and update exposure
            selling_row = df[
                (df["maturity_id"] == hkey[0]) &
                (df["date_id"] == iterdate) &
                (df["Strike"] == hkey[1]) &
                (df["Symbol"] == hkey[2])
            ]
            if not selling_row.empty and current_exposure_col in selling_row.columns:
                sell_dict[hkey] = min(selling_row["Bid Size"].values[0], holdings[hkey])
                exposure += selling_row[current_exposure_col].values[0] * (holdings[hkey] - sell_dict[hkey])
            else:
                sell_dict[hkey] = 0

    # If target exposure is reached, update holdings and compute PnL
    if exposure >= 100000:
        new_holdings = {}
        for key, sell_qty in sell_dict.items():
            if key in holdings:
                holdings[key] -= sell_qty
                if holdings[key] <= 0:
                    del holdings[key]
        for key, buy_qty in buy_dict.items():
            if key in holdings:
                holdings[key] += buy_qty
            else:
                holdings[key] = buy_qty

        for hkey in list(holdings.keys()):
            filtered_row = df[
                (df["maturity_id"] == hkey[0]) &
                (df["date_id"] == iterdate) &
                (df["Strike"] == hkey[1]) &
                (df["Symbol"] == hkey[2])
            ]
            if not filtered_row.empty and current_pnl_col_1 in filtered_row.columns:
                PnL += holdings[hkey] * filtered_row[current_pnl_col_1].values[0]

        for key, qty in holdings.items():
            maturity_id, strike, symbol = key
            next_key = (maturity_id, strike, symbol)
            if maturity_id > iterdate + 1:
                new_holdings[next_key] = qty  

        holdings = new_holdings.copy()
        daily_log = (f"Date: {iterdate}\n"
                    f"Buy orders: {buy_dict}\n"
                    f"Sell orders: {sell_dict}\n"
                    f"Exposure: {exposure *100}\n"
                    f"PnL: {PnL *100}\n")
        print(daily_log)
        daily_records.append(daily_log)
        total_PnL += PnL
        continue

    # If exposure is below target, continue buying options
    df_temp = df_date[iterdate].copy()
    # Predict priority using the trained model
    features = ['date_id', 'maturity_id', 'Symbol', 'Strike', 'Bid Price', 'Bid Size', 'Ask Price', 'Ask Size']
    df_temp["Symbol_enc"] = le.transform(df_temp["Symbol"])
    X_temp = df_temp[['date_id', 'maturity_id', 'Symbol_enc', 'Strike', 'Bid Price', 'Bid Size', 'Ask Price', 'Ask Size']].values
    X_temp_scaled = scaler.transform(X_temp)
    df_temp["priority"] = model.predict(X_temp_scaled).flatten()
    df_sorted = df_temp.sort_values(by="priority", ascending=True)

    for index, row in df_sorted.iterrows():
        key = (row["maturity_id"], row["Strike"], row["Symbol"])
        ask_size = row["Ask Size"]
        max_buy_qty = ask_size  # Maximum buy quantity is the Ask Size

        # If the option exists in sell_dict, offset the sell order first
        if key in sell_dict:
            additional_needed = 100000 - exposure
            if row[current_exposure_col] != 0:
                required_reduce_qty = int(np.ceil(additional_needed / row[current_exposure_col]))
            else:
                required_reduce_qty = 0
                continue
            if required_reduce_qty <= sell_dict[key]:
                reduce_qty = required_reduce_qty
                sell_dict[key] -= reduce_qty
                if sell_dict[key] == 0:
                    del sell_dict[key]
                exposure += reduce_qty * row[current_exposure_col]
                break
            else:
                reduce_qty = sell_dict[key]
                del sell_dict[key]
                exposure += reduce_qty * row[current_exposure_col]

        additional_needed = 100000 - exposure
        if row[current_exposure_col] != 0:
            required_buy_qty = int(np.ceil(additional_needed / row[current_exposure_col]))
        else:
            continue
        if required_buy_qty <= max_buy_qty:
            buy_qty = required_buy_qty
            exposure += buy_qty * row[current_exposure_col]
            buy_dict[key] = buy_dict.get(key, 0) + buy_qty
            break
        else:
            buy_qty = max_buy_qty
            exposure += buy_qty * row[current_exposure_col]
            buy_dict[key] = buy_dict.get(key, 0) + buy_qty

    # Update holdings with new trades
    new_holdings = {}
    for key, sell_qty in sell_dict.items():
        if key in holdings:
            holdings[key] -= sell_qty
            if holdings[key] <= 0:
                del holdings[key]
    for key, buy_qty in buy_dict.items():
        if key in holdings:
            holdings[key] += buy_qty
        else:
            holdings[key] = buy_qty

    # Calculate PnL for current day
    for hkey in list(holdings.keys()):
        filtered_row = df[
            (df["maturity_id"] == hkey[0]) &
            (df["date_id"] == iterdate) &
            (df["Strike"] == hkey[1]) &
            (df["Symbol"] == hkey[2])
        ]
        if not filtered_row.empty and current_pnl_col_1 in filtered_row.columns:
            PnL += holdings[hkey] * filtered_row[current_pnl_col_1].values[0]

    for key, qty in holdings.items():
        maturity_id, strike, symbol = key
        next_key = (maturity_id, strike, symbol)
        if maturity_id > iterdate + 1:
            new_holdings[next_key] = qty

    holdings = new_holdings.copy()
    daily_log = (f"Date: {iterdate}\n"
                 f"Buy orders: {buy_dict}\n"
                 f"Sell orders: {sell_dict}\n"
                 f"Exposure: {exposure *100}\n"
                 f"PnL: {PnL *100}\n")
    print(daily_log)
    daily_records.append(daily_log)
    total_PnL += PnL
    
total_log = f"Total PnL: {total_PnL *100}\n"
print(total_log)
daily_records.append(total_log)

with open("daily_trade_report.txt", "w") as f:
    f.write("\n".join(daily_records))

print("Daily trade report exported to 'daily_trade_report.txt'")




 === Date 1 ===
Starting new day, current holdings:
{}
33/33 [==============================] - 0s 454us/step
Date: 1
Buy orders: {(3, 485.0, 'QQQ'): 2444, (4, 562.0, 'SPY'): 3169, (4, 486.0, 'QQQ'): 1883, (3, 562.0, 'SPY'): 11564}
Sell orders: {}
Exposure: 10000324.738999953
PnL: -679078.9999999999


 === Date 2 ===
Starting new day, current holdings:
{(3, 485.0, 'QQQ'): 2444, (4, 562.0, 'SPY'): 3169, (4, 486.0, 'QQQ'): 1883, (3, 562.0, 'SPY'): 11564}
44/44 [==============================] - 0s 481us/step
Date: 2
Buy orders: {(4, 224.0, 'IWM'): 2581, (3, 472.0, 'QQQ'): 8966, (3, 221.0, 'IWM'): 128, (5, 556.0, 'SPY'): 6160, (3, 553.0, 'SPY'): 3164}
Sell orders: {(3, 485.0, 'QQQ'): 247, (4, 562.0, 'SPY'): 3169, (4, 486.0, 'QQQ'): 1883, (3, 562.0, 'SPY'): 6377}
Exposure: 10000229.907999994
PnL: -779789.0


 === Date 3 ===
Starting new day, current holdings:
{(4, 224.0, 'IWM'): 2581, (5, 556.0, 'SPY'): 6160}
51/51 [==============================] - 0s 472us/step
Date: 3
Buy orders: {(5, 